In [11]:
from itertools import count

import sympy
import numpy as np
import tinyarray as ta

from polynomial_orders_U import compute_next_orders, H_tilde
from sympy import MatrixSymbol

In [12]:
wanted_orders = [[4]]
Ns = [2, 2] # np.random.randint(1, high=5, size=2)


def hamiltonians(Ns, wanted_orders):
    """
    Produce random Hamiltonians to test.

    Ns: dimension of each block (A, B)
    wanted_orders: list of orders to compute

    Returns:
    hams: list of Hamiltonians
    """
    N_p = len(wanted_orders[0])
    orders = ta.array(np.eye(N_p))
    hams = []
    for i in range(2):
        hams.append(np.diag(np.sort(np.random.rand(Ns[i])) - i))

    def matrices_it(N_i, N_j, hermitian):
        """
        Generate random matrices of size N_i x N_j.

        N_i: number of rows
        N_j: number of columns
        hermitian: if True, the matrix is hermitian

        Returns:
        generator of random matrices
        """
        for i in count():
            H = np.random.rand(N_i, N_j) + 1j * np.random.rand(N_i, N_j)
            if hermitian:
                H += H.conj().T
            yield H

    for i, j, hermitian in zip([0, 1, 0], [0, 1, 1], [True, True, False]):
        matrices = matrices_it(Ns[i], Ns[j], hermitian)
        hams.append({order: matrix for order, matrix in zip(orders, matrices)})
    return hams



In [13]:
hams = hamiltonians(Ns, wanted_orders)
H_0_AA, H_0_BB, H_p_AA, H_p_BB, H_p_AB = hams
exp_S = compute_next_orders(*hams, wanted_orders=wanted_orders)

H_AA_01 = H_tilde(H_0_AA, np.zeros_like(H_0_BB), {}, {}, {}, wanted_orders, exp_S)[0]
H_AA_02 = H_tilde(H_0_AA, H_0_BB, {}, {}, {}, wanted_orders, exp_S)[0]
H_AA_p1 = H_tilde(np.zeros_like(H_0_AA), np.zeros_like(H_0_BB), H_p_AA, {}, {}, wanted_orders, exp_S)[0]
H_AA_p2 = H_tilde(np.zeros_like(H_0_AA), np.zeros_like(H_0_BB), {}, H_p_BB, {}, wanted_orders, exp_S)[0]
H_AA_p3 = H_tilde(np.zeros_like(H_0_AA), np.zeros_like(H_0_BB), {}, {}, H_p_AB, wanted_orders, exp_S)[0]

In [14]:
for order in H_AA_0:
    print(H_AA_0[order] / H_AA_p[order])
    print()
    print()


NameError: name 'H_AA_0' is not defined

In [ ]:
# test

dim1 = 2
dim2 = 2

H_0_AA = MatrixSymbol("{H_{0}^{AA}}", dim2, dim2)
H_0_BB = MatrixSymbol("{H_{0}^{BB}}", dim2, dim2)

H_p_AA = {}
H_p_BB = {}
H_p_AB = {}

H_p_AA[ta.array([1])] = MatrixSymbol("{H_{p}^{AA}}", dim1, dim1)
H_p_BB[ta.array([1])] = MatrixSymbol("{H_{p}^{BB}}", dim2, dim2)   
H_p_AB[ta.array([1])] = MatrixSymbol("{H_{p}^{AB}}", dim1, dim2)

In [ ]:
class EnergyDivider:
    def __init__(self):
        self.data = {}
        
    def __call__(self, rhs):
        new_entry = MatrixSymbol(f"V_{len(self.data) + 1}", dim1, dim2)
        self.data[new_entry] = rhs
        return new_entry

In [ ]:
wanted_orders=[[4]]
divider = EnergyDivider()
exp_S = compute_next_orders(
    H_0_AA, H_0_BB, H_p_AA, H_p_BB, H_p_AB,
    wanted_orders=wanted_orders,
    divide_energies=divider
)


In [ ]:
divider.data

{V_1: -{H_{p}^{AB}},
 V_2: V_1*{H_{p}^{BB}} - {H_{p}^{AA}}*V_1,
 V_3: (1/2)*V_1*Adjoint(V_1)*{H_{0}^{AA}}*V_1 + (1/2)*V_1*Adjoint(V_1)*{H_{p}^{AB}} + V_1*Adjoint({H_{p}^{AB}})*V_1 - (1/2)*V_1*{H_{0}^{BB}}*Adjoint(V_1)*V_1 + V_2*{H_{p}^{BB}} - {H_{p}^{AA}}*V_2 + (1/2)*{H_{p}^{AB}}*Adjoint(V_1)*V_1,
 V_4: -(-(1/2)*V_1*Adjoint(V_2) - (1/2)*V_2*Adjoint(V_1))*{H_{0}^{AA}}*V_1 - (-(1/2)*V_1*Adjoint(V_2) - (1/2)*V_2*Adjoint(V_1))*{H_{p}^{AB}} + (1/2)*V_1*Adjoint(V_1)*{H_{0}^{AA}}*V_2 + (1/2)*V_1*Adjoint(V_1)*{H_{p}^{AA}}*V_1 + V_1*Adjoint({H_{p}^{AB}})*V_2 + V_1*{H_{0}^{BB}}*(-(1/2)*Adjoint(V_1)*V_2 - (1/2)*Adjoint(V_2)*V_1) - (1/2)*V_1*{H_{p}^{BB}}*Adjoint(V_1)*V_1 + V_2*Adjoint({H_{p}^{AB}})*V_1 - (1/2)*V_2*{H_{0}^{BB}}*Adjoint(V_1)*V_1 + V_3*{H_{p}^{BB}} - {H_{p}^{AA}}*V_3 - {H_{p}^{AB}}*(-(1/2)*Adjoint(V_1)*V_2 - (1/2)*Adjoint(V_2)*V_1)}

In [ ]:
H_tilde_AA, H_tilde_BB = H_tilde(H_0_AA,
                                 H_0_BB,
                                 H_p_AA,
                                 H_p_BB,
                                 H_p_AB,
                                 wanted_orders,
                                 exp_S
                                )

In [ ]:
H_tilde_AA[(2,)]

-(1/2)*V_1*Adjoint(V_1)*Adjoint({H_{0}^{AA}}) + (1/2)*V_1*Adjoint({H_{0}^{BB}})*Adjoint(V_1) - V_1*Adjoint({H_{p}^{AB}}) + (1/2)*V_1*{H_{0}^{BB}}*Adjoint(V_1) - (1/2)*{H_{0}^{AA}}*V_1*Adjoint(V_1) - {H_{p}^{AB}}*Adjoint(V_1)

In [ ]:
list(divider.data.items())[0]

(V_1, -{H_{p}^{AB}})

In [ ]:
sympy.simplify(H_tilde_AA[(2,)].subs(*list(divider.data.items())[0][::-1]))

-(-V_1)*Adjoint(V_1) - V_1*Adjoint(-V_1) - (1/2)*V_1*Adjoint(V_1)*Adjoint({H_{0}^{AA}}) + (1/2)*V_1*Adjoint({H_{0}^{BB}})*Adjoint(V_1) + (1/2)*V_1*{H_{0}^{BB}}*Adjoint(V_1) - (1/2)*{H_{0}^{AA}}*V_1*Adjoint(V_1)

In [ ]:
[sympy.Equality(H_0_AA @ v - v @ H_0_BB, y) for v, y in divider.data.items()][0]

Eq(-V_1*{H_{0}^{BB}} + {H_{0}^{AA}}*V_1, -{H_{p}^{AB}})

In [ ]:
H_tilde_AA[(3,)]

(-(1/2)*V_1*Adjoint(V_2) - (1/2)*V_2*Adjoint(V_1))*Adjoint({H_{0}^{AA}}) - (1/2)*V_1*Adjoint(V_1)*Adjoint({H_{p}^{AA}}) + (1/2)*V_1*Adjoint({H_{p}^{BB}})*Adjoint(V_1) + V_1*{H_{0}^{BB}}*Adjoint(V_2) + (1/2)*V_1*{H_{p}^{BB}}*Adjoint(V_1) + V_2*Adjoint({H_{0}^{BB}})*Adjoint(V_1) - V_2*Adjoint({H_{p}^{AB}}) + {H_{0}^{AA}}*(-(1/2)*V_1*Adjoint(V_2) - (1/2)*V_2*Adjoint(V_1)) - (1/2)*{H_{p}^{AA}}*V_1*Adjoint(V_1) - {H_{p}^{AB}}*Adjoint(V_2)

In [ ]:
[sympy.Equality(H_0_AA @ v - v @ H_0_BB, y) for v, y in divider.data.items()][1]

Eq(-V_2*{H_{0}^{BB}} + {H_{0}^{AA}}*V_2, V_1*{H_{p}^{BB}} - {H_{p}^{AA}}*V_1)

In [ ]:
H_tilde_AA[(4,)]

(-(1/2)*V_1*Adjoint(V_2) - (1/2)*V_2*Adjoint(V_1))*Adjoint({H_{p}^{AA}}) + (-(1/4)*Adjoint((-(1/2)*V_1*Adjoint(V_1))**2) - (1/4)*(-(1/2)*V_1*Adjoint(V_1))**2 - (1/2)*V_1*Adjoint(V_3) - (1/2)*V_2*Adjoint(V_2) - (1/2)*V_3*Adjoint(V_1))*Adjoint({H_{0}^{AA}}) + (1/8)*V_1*Adjoint(V_1)*Adjoint({H_{0}^{AA}})*V_1*Adjoint(V_1) + (1/8)*V_1*Adjoint(V_1)*{H_{0}^{AA}}*V_1*Adjoint(V_1) + (1/2)*V_1*Adjoint(V_1)*{H_{p}^{AB}}*Adjoint(V_1) + (1/2)*V_1*Adjoint({H_{p}^{AB}})*V_1*Adjoint(V_1) + V_1*{H_{0}^{BB}}*Adjoint(V_3) + V_1*{H_{p}^{BB}}*Adjoint(V_2) + (1/2)*V_2*Adjoint({H_{0}^{BB}})*Adjoint(V_2) + V_2*Adjoint({H_{p}^{BB}})*Adjoint(V_1) + (1/2)*V_2*{H_{0}^{BB}}*Adjoint(V_2) + V_3*Adjoint({H_{0}^{BB}})*Adjoint(V_1) - V_3*Adjoint({H_{p}^{AB}}) + {H_{0}^{AA}}*(-(1/4)*Adjoint((-(1/2)*V_1*Adjoint(V_1))**2) - (1/4)*(-(1/2)*V_1*Adjoint(V_1))**2 - (1/2)*V_1*Adjoint(V_3) - (1/2)*V_2*Adjoint(V_2) - (1/2)*V_3*Adjoint(V_1)) + {H_{p}^{AA}}*(-(1/2)*V_1*Adjoint(V_2) - (1/2)*V_2*Adjoint(V_1)) - {H_{p}^{AB}}*Adjoint(V

In [ ]:
[sympy.Equality(H_0_AA @ v - v @ H_0_BB, y) for v, y in divider.data.items()][2]

Eq(-V_3*{H_{0}^{BB}} + {H_{0}^{AA}}*V_3, (1/2)*V_1*Adjoint(V_1)*{H_{0}^{AA}}*V_1 + (1/2)*V_1*Adjoint(V_1)*{H_{p}^{AB}} + V_1*Adjoint({H_{p}^{AB}})*V_1 - (1/2)*V_1*{H_{0}^{BB}}*Adjoint(V_1)*V_1 + V_2*{H_{p}^{BB}} - {H_{p}^{AA}}*V_2 + (1/2)*{H_{p}^{AB}}*Adjoint(V_1)*V_1)

In [ ]:
list(divider.data.values())[1]

V_1*{H_{p}^{BB}} - {H_{p}^{AA}}*V_1

In [ ]:
[sympy.Equality(H_0_AA @ v - v @ H_0_BB, y) for v, y in divider.data.items()][0]

Eq(-V_1*{H_{0}^{BB}} + {H_{0}^{AA}}*V_1, -{H_{p}^{AB}})